In [33]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import nltk

import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [34]:
trainData = pd.read_csv(filepath_or_buffer='data/train.dat.txt', header=None, sep='\t',names=["rating","review"])[0:2]
testData =  pd.read_csv(filepath_or_buffer='data/test.dat.txt', header=None,sep='\n',names=["review"])[0:2]


In [35]:
def preProcess(data):
    print(data)
    data["review"] = [BeautifulSoup(text).get_text() for text in data["review"] ]
    data["review"] = data["review"].map(lambda x: re.sub(r'\W+', " ", x))
    data["review"] = data["review"].map(lambda x: re.sub(r'\d+', "", x))
    data["review"] = data["review"].map(lambda x: WordNetLemmatizer().lemmatize(x))
    data["review"] = data["review"].map(lambda x: PorterStemmer().stem(x))
    data["review"] = data["review"].str.lower().str.split(" ")
    
    stop_words = stopwords.words('english')
    data["review"] = data["review"].apply(lambda x: [item for item in x if item not in stop_words])
    return data;
    
trainData = preProcess(trainData)
testData = preProcess(testData)
print('pre-processing done')

   rating                                             review
0      -1  Although a film with Bruce Willis is always wo...
1      -1  This movie was slower then Molasses in January...
                                              review
0  I am so glad when i watch in every time the mo...
1  when I first heard about this movie, I noticed...
pre-processing done


In [36]:
idx = {}
tid = 0
nnz = 0
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    global idx
    global tid
    global nnz
    nrows = len(docs)
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
    return ncols,nrows

def combine_matrix(ncols, nrows, docs):
        
    # set up memory
    global nnz
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    global idx
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
    
    mat = csr_matrix((val, ind, ptr), shape=(nrows, nnz), dtype=np.double)
    mat.sort_indices()
    return mat

ncols1, nrows1 = build_matrix(trainData["review"])
ncols2, nrows2 = build_matrix(testData["review"])

trainCSR = combine_matrix(ncols1, nrows1, trainData["review"])
testCSR = combine_matrix(ncols2, nrows2, testData["review"])

In [37]:
def csr_l2normalize(mat,nnz, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    nrows = mat.shape[0]
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
    return csr_matrix((val, ind, ptr), shape=(nrows, nnz), dtype=np.double)
global normTrain
global normTest

normTrain = csr_l2normalize(trainCSR, nnz)
normTest = csr_l2normalize(testCSR, nnz)

In [38]:
def classify(x, train, clstr, k=17):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]

In [39]:
try:
    outFile = open("data/output.dat", 'w')
    l = [ classify(normTest[i], normTrain, trainData["rating"]) for i in range(normTest.shape[0]) ]
    for i in range(len(l)):
        if str(l[i])=="+1":
            outFile.write("+1\n")
        elif str(l[i])=="-1":
            outFile.write("-1\n")
except IOError as (errno, strerror):
     print "I/O error({0}): {1}".format(errno, strerror)
finally:
    outFile.close()

ValueError: I/O operation on closed file